### Audio

In [15]:
from joblib import load
import librosa
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
# Load the best model from the file
audio_loaded_model = load('Final_Audio_Model.joblib')


In [4]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad']

In [11]:
label_encoder = LabelEncoder()
scaler = StandardScaler()

In [ ]:
# Extract more audio features
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path)
    except Exception as e:
        print(f"Error loading file: {file_path} - {str(e)}")
        return None
    
    if file_path.lower().endswith(".wav"):
        features = [
            np.mean(librosa.feature.chroma_stft(y=y, sr=sr)),
            np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
            np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
            np.mean(librosa.feature.mfcc(y=y, sr=sr)),
            np.mean(librosa.feature.zero_crossing_rate(y)),
            np.mean(librosa.feature.spectral_contrast(y=y, sr=sr)),
            np.mean(librosa.feature.tempogram(y=y, sr=sr))
        ]
        return features
    else:
        return None

In [16]:
# Extract audio features and labels
# Define the root directory where your audio files are stored
# Create lists to store features and labels
X = []
y = []
root_dir = "Audio(OLD)"
for emotion in emotions:
    folder_path = os.path.join(root_dir, emotion)
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            file_path = os.path.join(folder_path, filename)
            features = extract_features(file_path)
            if features is not None:
                X.append(features)
                y.append(emotion)


In [17]:
# Split the data into training and testing sets
# Convert the feature and label lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Encode emotion labels using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)

In [18]:

def predict_emotions(audio_path, audio_loaded_model, label_encoder):
    try:
        y, sr = librosa.load(audio_path)
    except Exception as e:
        print(f"Error loading audio: {str(e)}")
        return None

    # Extract audio features from the input audio
    features = extract_features(audio_path)
    if features is not None:
        # Standardize the features
        features = scaler.transform([features])
        
        # Predict the emotion probabilities using the loaded model
        emotion_probabilities = audio_loaded_model.predict_proba(features)
        
        # Get the top 3 emotions with highest probabilities
        top_emotions_indices = (-emotion_probabilities).argsort()[0][:3]
        
        top_emotions = label_encoder.inverse_transform(top_emotions_indices)
        top_probabilities = [emotion_probabilities[0][index] for index in top_emotions_indices]
        
        return top_emotions, top_probabilities
    else:
        return None

# Provide an audio path to predict emotions
audio_path = "RAVDEES_AUDIO\\Disgust\\03-01-07-01-01-01-04.wav"
predicted_emotions, probabilities = predict_emotions(audio_path, audio_loaded_model, label_encoder)

if predicted_emotions is not None:
    print("Predicted Emotions:")
    for emotion, probability in zip(predicted_emotions, probabilities):
        print(f"{emotion}: {probability:.2f}")
else:
    print("No emotions predicted.")


Predicted Emotions:
Disgust: 0.71
Happy: 0.09
Angry: 0.06


## Image

In [19]:
import numpy as np
from PIL import Image
from keras.models import load_model
import os


In [23]:
# Load the saved model
image_loaded_model = load_model('Final_Image_Model.h5')

In [24]:
# Define the class names for your dataset
class_names = ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad']


In [25]:
# Load and preprocess the input image
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((80, 80))  # Resize the image to the expected dimensions
    img = img.convert('RGB')  # Ensure the image has 3 color channels (RGB)
    img_array = np.array(img) / 255.0  # Normalize the image data
    img_array = img_array.reshape(1, 80, 80, 3)  # Reshape for the model
    return img_array

# Replace 'input_image.jpg' with the path to your input image
input_image_path = 'angry_image.png'

# Load and preprocess the input image
input_data = load_and_preprocess_image(input_image_path)

# Make predictions on the input image
predictions = image_loaded_model.predict(input_data)

# Get the predicted class label
predicted_class_index = np.argmax(predictions)
predicted_class = class_names[predicted_class_index]

# Print the predicted class and confidence scores
print(f'Predicted Emotion: {predicted_class}')
print('Confidence Scores:')
for i, class_name in enumerate(class_names):
    confidence = predictions[0][i]
    print(f'{class_name}: {confidence:.2f}')


1/1 [==============================] - 0s 244ms/step
Predicted Emotion: anger
Confidence Scores:
anger: 0.48
disgust: 0.02
fear: 0.43
happy: 0.01
neutral: 0.05
sad: 0.02


## Text

In [16]:
import joblib
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

In [2]:
# Load the saved model
text_loaded_model = joblib.load('Final_Text_Model.pkl')

In [13]:
# Specify the correct encoding for your CSV file
encoding = 'ISO-8859-1'  # Replace with the appropriate encoding

# Read the CSV file with the specified encoding
data = pd.read_csv('Text Data.csv', encoding=encoding)

In [14]:
# Separate the dataset by emotion
happy = data[data['EMOTIONS'] == 'Happy']
neutral = data[data['EMOTIONS'] == 'Neutral']
sad = data[data['EMOTIONS'] == 'Sad']
anger = data[data['EMOTIONS'] == 'Anger']
fear = data[data['EMOTIONS'] == 'Fear']
disgust = data[data['EMOTIONS'] == 'Disgust']

In [17]:
# Upsample minority classes to match the majority class (Happy)
resampled_neutral = resample(neutral, replace=True, n_samples=len(happy), random_state=42)
resampled_sad = resample(sad, replace=True, n_samples=len(happy), random_state=42)
resampled_anger = resample(anger, replace=True, n_samples=len(happy), random_state=42)
resampled_fear = resample(fear, replace=True, n_samples=len(happy), random_state=42)
resampled_disgust = resample(disgust, replace=True, n_samples=len(happy), random_state=42)


In [18]:
# Combine the resampled dataframes
balanced_data = pd.concat([happy, resampled_neutral, resampled_sad, resampled_anger, resampled_fear, resampled_disgust])

# Shuffle the dataset
data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [19]:

# Split data into input (X) and target (y)
X = data['TEXT'].values
y = data['EMOTIONS'].values

# Encode emotion labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
max_sequence_length = 50 

In [21]:
# Assuming you have loaded the model using joblib as 'loaded_model'

# Example text input
new_text = "Would feel weird swiping right on a leftie"

# Tokenize and pad the new text
new_text_seq = tokenizer.texts_to_sequences([new_text])
new_text_pad = pad_sequences(new_text_seq, maxlen=max_sequence_length, padding='post')

# Make a prediction using the loaded model
predicted_probabilities = text_loaded_model.predict_proba(new_text_pad)

# Get the top three emotions and their probabilities
top_emotions_indices = predicted_probabilities.argsort()[0][-3:][::-1]
top_emotions = label_encoder.inverse_transform(top_emotions_indices)
top_probabilities = predicted_probabilities[0][top_emotions_indices]

# Print the top three predicted emotions and their probabilities
for emotion, probability in zip(top_emotions, top_probabilities):
    print(f"{emotion}: {probability:.4f}")


Disgust: 0.7000
Neutral: 0.3000
Sad: 0.0000
